# BERT and Feature Extraction ( теория + практика )

Слайды про модели основанные на архитектуре трансформер (с механизмом внимания): https://docs.google.com/presentation/d/1Kpk3DcHNf1iOMcggF701VorNuI3csqcuU9G69mUxtZU/edit#slide=id.p

Feature extraction - выделения празнаков на примере создания вектора описания (дескриптора или эмбеддинга) текста с помощью нейросетевой модели.

Полученный вектор представляет смысл текста. Этот вектор далее можно использовать для задач классфикации текстов, сравнения текстов, кластеризации текстов и т.д.

#### Установка библиотек

In [ ]:
# пакеты transformers c удобным API для работы с моделями, основанными на трансформерах
%pip install transformers
%pip install datasets tokenizers

# удобный пакет для получения представлений текстов (рекомендуется для текстов)
%pip install sentence_transformers
# пакеты Hugging Face
# https://huggingface.co/docs/transformers/index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

#### Подключение библиотек

In [ ]:
# torch (pytorch) - пакет для создания нейросетей
import torch

# классы для загрузки моделей токенизации и языковых моделей
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
# https://huggingface.co/docs/datasets/index

# функция tqdm из пакета tqdm может выводить полосу прогресса в циклах,
# вычислять ожидаемое время выполнения цикла
from tqdm import tqdm

# is_available() вернёт True, если видеокарта доступна
Device = "cuda" if torch.cuda.is_available() else "cpu"
Device
# переменная Device будет использоваться для обозначения устройства, на котором нужно выполнять код
# и памятью которого нужно пользоваться
# cuda - хранить данные в оперативной памяти видеокарты и выполнять код на GPU
# cpu - хранить данные в оперативной памяти и выполнять код на CPU

'cuda'

## Модели типа Sentence Transformer

Лидерборд моделей:
- https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
from sentence_transformers import SentenceTransformer

# скачивание (при необходимости) предвариетльно обученной модели типа LaBSE
# LaBSE - language-agnostic BERT sentence embedding - независимый от языка BERT для создания представлений (эмбеддингов) текста
# BERT - Bidirectional Encoder Representations from Transformers
sen_tra = SentenceTransformer("cointegrated/LaBSE-en-ru").to(Device)
# см каталог подобных моделей  (с тегом feature extraction) на https://huggingface.co
# https://huggingface.co/ai-forever/sbert_large_nlu_ru - ai-forever/sbert_large_nlu_ru

sen_tra
# SentenceTransformer состоит из нескольких блоков:
# - непосредственно трансформер, принимает текст, токнизирует, прогоняет через блоки трансформера
# - слой pooling, создаёт вектор из небольшого числа компонент из выхода трансформера
# - полносвязный слой, создаёт окончательное векторное представление текста - эмбеддинг текста
# - слой нормализации выхода

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

In [ ]:
# сразу видны важные характеристики модели:
# 'max_seq_length' - максимальная длина входной последовательности в токенах
# на последнем слое 'out_features' - размерность вектора, представляющего текст

In [ ]:
# характеристики модели

# print(sen_tra) # трансформер
# print(sen_tra[1]) # блок преобразования выхода трансформера в эмбеддинг текста

# максимальная длина последовательности в токенах
max_length = sen_tra.tokenizer.model_max_length
# размерность вектора эмбеддинга текста
embedding_dim = sen_tra.get_sentence_embedding_dimension()
# Get model configuration
config = sen_tra[0].auto_model.config
# Количество слоёв (блоков)
num_layers = config.num_hidden_layers
# размер вектора, представляющего каждый токена
hidden_size = config.hidden_size
# Количество голов внимания (столько голов внимания в каждом слое (блоке трансформера)
headsnum_attention_heads = config.num_attention_heads
# Размер словаря
vocab_size = config.vocab_size
print( f"Количество параметров языковой модели: {sum( p.numel() for p in sen_tra.parameters() if p.requires_grad):,}")
print(f"num_layers: {num_layers}")
print(f"headsnum_attention_heads: {headsnum_attention_heads}")
print(f"vocab_size: {vocab_size:7,d}")

# характеристики блока, отвечающего за создание вектора текста из ответа нейросети (encoder'а)
pooling_config = sen_tra[1].get_config_dict()

# словарь токенизатора
vocab = sen_tra.tokenizer.get_vocab()
# обозначения специальных токенов
special_tokens = sen_tra.tokenizer.special_tokens_map

Количество параметров языковой модели: 128,935,680
num_layers: 12
headsnum_attention_heads: 12
vocab_size:  55,083


In [ ]:
# Пример использования токенизатора и модели
# токенизация текста - разделение текста на отдельные слова или их части, имеющие смысл для модели
# как правило в процессе токенизации токены заменяется и на их номера их словаря (хотя можно посмотреть и слова\части слов)

# примеры текстов для токенизации
sentences = ["Hello World", "World domination", "бетономешалка мешает бетон", "бетономешалка", "машинное обучение", "Exterminate!"]

In [ ]:
# пример токенизации одного текста
sen_tra.tokenizer.tokenize( sentences[2] )

['бетон', '##оме', '##шал', '##ка', 'мешает', 'бетон']

In [ ]:
# токенизация текста и преобразование в индексы токенов
sen_tra.tokenizer( sentences )
# перед обработкой текстов рекомендуется оценить их длины в токенах. все ли тексты поместятся в трансформер?

{'input_ids': [[2, 6467, 1587, 3], [2, 1587, 20529, 1489, 3], [2, 19302, 24500, 34169, 940, 35445, 19302, 3], [2, 19302, 24500, 34169, 940, 3], [2, 8836, 2759, 11914, 3], [2, 41374, 49351, 622, 5, 3]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
# словарь токенов
sen_tra.tokenizer.get_vocab()

{'neighbors': 30600,
 'bedrooms': 22692,
 '##ball': 7231,
 'Janis': 38848,
 'Breed': 47197,
 '##To': 51274,
 'circuits': 25691,
 'looked': 9401,
 'conclu': 45079,
 'MSG': 23792,
 '##овна': 20941,
 'alley': 50418,
 'Peter': 2049,
 'Nicaragua': 19497,
 'BI': 9868,
 'darkness': 31273,
 '##бива': 33647,
 'лучше': 4216,
 'peculiar': 33516,
 'беременность': 37450,
 '##ыб': 45795,
 'HIGH': 11326,
 '##aki': 7256,
 'мероприятии': 46111,
 'Cavalier': 49484,
 'Tactics': 39330,
 'необходимыми': 48419,
 '##зна': 13030,
 'A7': 15747,
 'оценку': 33617,
 'Николов': 34347,
 '##ко': 2201,
 'Dancing': 20435,
 '##ующий': 38637,
 '##RUM': 47937,
 'заявили': 24573,
 '##ничать': 46914,
 '##очок': 47220,
 'responded': 29473,
 'Город': 16693,
 'avoid': 8654,
 'сколько': 8883,
 'invoice': 27275,
 'MAD': 32988,
 '##рир': 41309,
 'CUP': 23026,
 'MCM': 44724,
 'побережье': 40744,
 '7800': 48341,
 'схему': 25741,
 'нефти': 27796,
 'SSI': 29999,
 '##BEL': 43715,
 'Человек': 23328,
 'FIX': 43100,
 'состоянии': 10580,

In [ ]:
# пример применения модели - создания векторных представлений переданных текстов
with torch.no_grad():
    # encode принимает сразу набор текстов, можно делить на батчи
    sen_emb = sen_tra.encode( sentences )

# векторные представления  (эмбеддинги), по умолчанию имеют тип numpy array
sen_emb.shape

(6, 768)

In [ ]:
# векторные представления текстов
sen_emb

array([[-0.07142265, -0.07716199, -0.03047758, ...,  0.01356032,
        -0.04016103, -0.02446148],
       [-0.05577283, -0.04117677,  0.04145663, ..., -0.01717833,
        -0.00162299, -0.06026322],
       [ 0.01279573, -0.01244833, -0.01031378, ...,  0.03337278,
         0.06542329,  0.04024685],
       [ 0.04310013, -0.01668501, -0.03217231, ...,  0.0146694 ,
         0.01792692,  0.04902544],
       [ 0.01399165, -0.02388861, -0.03167713, ..., -0.04918618,
        -0.01697674,  0.02499205],
       [-0.0148802 , -0.05789527,  0.0443352 , ..., -0.03850351,
         0.00706479, -0.01793129]], dtype=float32)

### Векторные представления на плоскости

Для примера отобразим векторные представления текстов на плоскости.
Так можно убедиться, с поправкой на потерю данных во время преобразования многомерного вектора в двумерный, что различные по смыслу тексты представлены различными точками на плоскости.

Точки - это окончания векторов, если начала всех векторов расположить в начале координат.

In [ ]:
# Для примера преобразуем многомерные вектора представлений текстов к двумерным,
# чтобы на плоскости (с потерей информации) оценить как они друг от друга отличаются

import plotly.express as px

# PCA - метод главных компонент, выполняет линейное преобразование многомерное пространство в пространство с меньшей размерностью (с потерей информации)
from sklearn.decomposition import PCA

reduce2d  = PCA(n_components=2)
X2d = reduce2d.fit_transform(sen_emb)
# cpu() - перенос переменной в оперативную память

# вычислений доли сохранившейся дисперсии (можно примерно понимать как долю сохранившейся информации)
print(f"explained variance: {reduce2d.explained_variance_ratio_.sum():4f}")

# построение точек на плоскости, на которые указывают вектора слов
px.scatter( title = 'Представления текстов',
            x=X2d[:,0], y=X2d[:,1],
            hover_data=[sentences],             # что показывать при наведении
            width=700)

explained variance: 0.630056


## Модели из пакета Transformers

Аналогичный пример получения векторных представлений, но через более громоздкий API пакета Transformers

In [ ]:
# https://huggingface.co/cointegrated/LaBSE-en-ru
# Language-agnostic BERT Sentence Embedding

TokenizerLBSE = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")

ModelLBSE =     AutoModel.from_pretrained("cointegrated/LaBSE-en-ru").to(Device)                   # предварительно обученный BERT




In [ ]:
ModelLBSE
# Модель состоит из трёх частей:
# 1. embeddings. принимает токенизированный текст (токен = idx); превращает токены в вектора размерностью 768;
# размер словаря - 55083 токенов (слов и частей слов)
# максимальное количество токенов - 512
# 2. encoder (12 слоёв)
# 3. pooler

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(55083, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# подсчёт параметров по всем слоям нейросети
print(f"parameters count: {sum( p.numel() for p in ModelLBSE.parameters() if p.requires_grad):,}")
# для сравнения, GPT2 - 1.5e9; GPT3 - 175e9; GPT4 - 1.76e12
# в целях демонстрации, ускорения вычислений и минимизации занимаемой памяти выбрана небольшая модель

parameters count: 128,345,088


### Пример применения токенизатора и модели

In [ ]:
# Пример использования токенизатора и модели
# токенизация текста - разделение текста на отдельные слова или их части, имеющие смысл для модели
# как правило в процессе токенизации токены заменяется и на их номера их словаря (хотя можно посмотреть и слова\части слов)

# примеры текстов для токенизации
sentences = ["Hello World",
             "World domination",
             "бетономешалка мешает бетон",
             "бетономешалка",
             "машинное обучение",
             "Exterminate!"]

# токенизация:
encoded_input = TokenizerLBSE(sentences,
                              padding=True,         # выравнивание длин последовательностей по длине максимальной последовательности
                              truncation=True,      # обрезать длинные тексты
                              max_length=200,       # максимальная длина текста в токенах
                              return_tensors='pt'   # возвращать данные в виде pytorch tensor
                              )
# encoded_input - словарь
print(encoded_input['input_ids'])       # токенизированный текст в виде номеров токенов
print(encoded_input['attention_mask'])
# attention_mask - задаёт маску для механизма внимания, игнорирует токены-заполнители (pad tokens)
# выход - номера токенов, в модели они будут преобразованы в векторы

# в начале предложения идёт служебный токен 2, обозначающий начало предложения, в конце 3 обозначает конец предложения

tensor([[    2,  6467,  1587,     3,     0,     0,     0,     0],
        [    2,  1587, 20529,  1489,     3,     0,     0,     0],
        [    2, 19302, 24500, 34169,   940, 35445, 19302,     3],
        [    2, 19302, 24500, 34169,   940,     3,     0,     0],
        [    2,  8836,  2759, 11914,     3,     0,     0,     0],
        [    2, 41374, 49351,   622,     5,     3,     0,     0]])
tensor([[1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0]])


In [ ]:
# пример применения модели - создания векторных представлений переданных текстов
with torch.no_grad():
    model_output = ModelLBSE(**encoded_input.to(Device))
    # **encoded_input - распаковка словаря в соответствующие параметры
    # аналогично:
    # model_output = ModelLBSE(input_ids = encoded_input['input_ids'].to(Device), attention_mask= encoded_input['attention_mask'].to(Device))
    # .to(Device) - перенос переменной в память устройства, указанного в Device

In [ ]:
# векторы размерности 768, представляющие тексты
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings)
# Эти векторы можно использовать вместо текстов, например в задаче классификации
# Можно дополнить модель новыми слоями нейронов для определения класса

tensor([[-0.0714, -0.0772, -0.0305,  ...,  0.0136, -0.0402, -0.0245],
        [-0.0558, -0.0412,  0.0415,  ..., -0.0172, -0.0016, -0.0603],
        [ 0.0128, -0.0124, -0.0103,  ...,  0.0334,  0.0654,  0.0402],
        [ 0.0431, -0.0167, -0.0322,  ...,  0.0147,  0.0179,  0.0490],
        [ 0.0140, -0.0239, -0.0317,  ..., -0.0492, -0.0170,  0.0250],
        [-0.0149, -0.0579,  0.0443,  ..., -0.0385,  0.0071, -0.0179]],
       device='cuda:0')


In [ ]:
embeddings.shape

torch.Size([6, 768])

In [ ]:
# Для примера преобразуем многомерные вектора представлений текстов к двумерным,
# чтобы на плоскости (с потерей информации) оценить как они друг от друга отличаются

import plotly.express as px

# PCA - метод главных компонент, выполняет линейное преобразование многомерное пространство в пространство с меньшей размерностью (с потерей информации)
from sklearn.decomposition import PCA

reduce2d  = PCA(n_components=2)
X2d = reduce2d.fit_transform(embeddings.cpu())
# cpu() - перенос переменной в оперативную память

# вычислений доли сохранившейся дисперсии (можно примерно понимать как долю сохранившейся информации)
print(f"explained variance: {reduce2d.explained_variance_ratio_.sum():4f}")

# построение точек на плоскости, на которые указывают вектора слов
px.scatter( title = 'Представления текстов',
            x=X2d[:,0], y=X2d[:,1],
            hover_data=[sentences],             # что показывать при наведении
            width=700)

explained variance: 0.630056


# Пример решения задачи классификации текстов (практика)

Решим задачу классификации текстов на примере негативных \ позитивных отзывов с сайта IMDB.

Схема метода:

1. **Извлечение признаков**. Текст --> [ токенизатор, замена токенов на из номера ] --> Номера токенов\
Номера токенов-->  [языковая модель создающая векторные представления текста] --> 768-мерный вектор текста

2. **Классификация** текста любым классификатором, но вместо текста используется его 768-мерный вектор.
Классификатором может выступать любая модель.

In [ ]:
dataset = load_dataset("imdb").shuffle(seed=42)      # отзывы с IMDB: https://huggingface.co/datasets/imdb
# 2 класса - негативный и позитивный

N = 500        # для примера отберём первые N текстов, иначе преобразование может занять много времени
TrainText = dataset['train']['text'][:N]
TrainY = dataset['train']['label'][:N]
TestText =  dataset['test']['text'][:N]
TestY = dataset['test']['label'][:N]

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/s/.cache/huggingface/datasets/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Fri Dec 27 12:54:36 2024).


In [ ]:
# пример текста
print( TrainText[0] )

# класс: ( 1 - позитивный обзор, 0 - негативный обзор)
print( TestText[0] )

In [ ]:
# todo: проверить баланс классов
# todo: изучить длины текстов
# todo: очистить текст от артефактов

In [ ]:
EMB_DIM = 768       # размерность вектора представляющего текст
# todo: получать эту размерность из трансформера

MAX_LEN = 211       # максимальная длинна текста в токенах (небольшая длина выбрана чтобы ускорить выполнение демонстрационного кода, чтобы не потреблять много памяти)
# todo: токенизировать часть выборку из текста чтобы понять какие длины имеют отзывы
# todo: получать эту длину из трансформера

# матрица для хранения векторных представлений текстов
# TrainX = torch.zeros( ( len( TrainText ), EMB_DIM) )

In [ ]:
help( sen_tra.encode)

In [ ]:
import gc
gc.collect()

3847

In [ ]:
# создадим векторные представления текстов


TrainX = sen_tra.encode( TrainText, batch_size = 128, show_progress_bar = True, precision="int8", convert_to_tensor=True)
# todo: тут можно преобразовать сразу все данные (треин и тест), т.к. на этом этапе не происходит обучения
# чтобы ускорить работу трансформера и уменьшить потребление памяти, можно ограничить точность (квантизивать) результирующие вектора
# precision = "float32" (исходная точность), "int8", "uint8", "binary", "ubinary";
# output_value='pt' - возвращаемый тип данных (pt =  )

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
TestX = sen_tra.encode( TestText, batch_size = 128, show_progress_bar = True, precision="int8", convert_to_tensor=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
TrainX

In [ ]:
torch.save(TrainX, 'imdb_dataset_trainx.pt')       # pt = pytorch tensor
torch.save(TestX,  'imdb_dataset_testx.pt')       # pt = pytorch tensor

In [ ]:
# сожмём файл чтобы было удобнее скачивать и хранить
# !zip imdb_dataset.pt.zip imdb_dataset.pt
!7z a -mx=9 imdb_dataset_trainx.pt.7z imdb_dataset_trainx.pt

In [ ]:
# загрузка текстового представления из файла
TrainX = torch.load('imdb_dataset_trainx.pt')
TestX = torch.load('imdb_dataset_testx.pt')
# совместимость между функциями save и load для разных версий torch не гарантируется,
# поэтому можно сохранять тензоры и с помошью pandas, numpy или др.

In [ ]:
TrainX

tensor([[  3,   0, -87,  ...,  78, -57,  89],
        [ -5, -43,  -5,  ..., -16,  25, 101],
        [  9,  18,  -6,  ...,   1, -86, -15],
        ...,
        [  0, -84,  27,  ...,   4, -20,  15],
        [ 37, -33, -39,  ...,  32, -73, -64],
        [ -3, -21, -24,  ..., -27,   1, -61]], dtype=torch.int8)

# Обучение модели для классификации текстов

## SVC

In [ ]:
from sklearn.svm import SVC

clf = SVC(random_state = 0, verbose=1)
clf.fit(TrainX, TrainY)

[LibSVM]*.*
optimization finished, #iter = 501
obj = -200.543176, rho = 0.274692
nSV = 409, nBSV = 211
Total nSV = 409


SVC(random_state=0, verbose=1)

### Оценка качества модели

In [ ]:
from sklearn.metrics import classification_report

# на тренировочных данных
y_train_pred = clf.predict( TrainX )
print(classification_report( TrainY, y_train_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       254
           1       0.98      0.98      0.98       246

    accuracy                           0.98       500
   macro avg       0.98      0.98      0.98       500
weighted avg       0.98      0.98      0.98       500



In [ ]:
# на тестовых данных
y_pred = clf.predict( TestX )
print(classification_report( TestY, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.78      0.79       254
           1       0.78      0.80      0.79       246

    accuracy                           0.79       500
   macro avg       0.79      0.79      0.79       500
weighted avg       0.79      0.79      0.79       500



In [ ]:
# TODO: применить более сложные модели для классификации
# TODO: дообучить (fine tune) модель ModelLBSE для данной задачи классификации
#

## Как решить задачу классификации текста?
1. Использовать статистические представления текста вместо представлений, полученных на основе языковой модели
    1. Кодирование текста:
        - BagOfWords
        - TF-IDF, [Объяснение метода и пример](https://colab.research.google.com/drive/1yOnvYUbbu7b2sgnh4vn1csis9PWAss_f#scrollTo=Jg_RRVV2-mhF)
    1. Классификация: использовать любую модель классификации
2. Нейросетевая модель на основе рекуррентных нейронов (RNN, LSTM, GRU).
    1. Кодирование текста:
        - Наивный подход: Замена слов на номера
        - Словарь или слой нейросети для создания векторных представлений слов (word2vec или др.)
    2. Классификация: подавать последовательности из номеров или векторов в нейросеть, которая будет классифицировать тексты.
3. Использовать предварительно обученную языковую модель, но предназначенную для классификации текстов. (модели на основе BERT, GPT и др.) Дообучить часть этой модели, которая отвечает только за классификацию.

4. Использовать предварительно обученную языковую модель, но предназначенную для создания векторных представлений текстов.
    1. Кодирование текста: языковая модель (BERT, LaBSE, Longformer и др.) -> вектор текста (типичная размерностью - несколько сотен)
    2. Классификация: использовать любую модель классфиикации

5. Случай длинных текстов.
Разбить текст на части (***чанки***).
    1. Кодирование: языковая модель, но текст разбит на части -> набор векторов текста
    2. Классификация (вариант 1):
        - Комбинация векторов текста (например среднее арифметическое, конкатенация) -> вектор всего текста
        - любая модель классификации
    2. Классификация (вариант 2):
        - любая модель классификации, способная принимать последовательность из векторов (например нейросеть на основе RNN, LSTM, GRU)
    3. ***Абстрактивная суммаризация***: суммаризировть текст с помощью LLM (например llama3 или специальный суммаризатор). Полученное краткое содержание уже можно подавать в модель sentence embeddings.    


6. Использовать тематическое моделирование: создать модель, кластеризующую тексты по темам. Использовать темы, к которым принадлежит текст как признаки.
Далее использовать любой классификатор.

# Другие варианты применения модели выдающей представления для текстов
- Выдача рекомендаций текстового контента на основе схожести представлений данного текста и остальных
    - Посты в социальной сети, новости на сайте или на платформе для блогов
    - Рекомендация похожих вопросов-ответов на сайтах типа stackoverflow
    - Поиск схожих по тема научных статей
- Кластеризации текстов с последующим выделением тем
- Фильтрация контента
- ...

# Ссылки

- Massive Text Embedding Benchmark (MTEB) Leaderboard - https://huggingface.co/spaces/mteb/leaderboard